# Show your Datascience Skills! :brain:

Use your datascience expertise to:
* Explore the existing Data
* Develop and Register new Features in the **Feature Store**
* Train and Register a Machine Learning Model in the **Model Registry**
* Create Scores for passengers with unknown survival status

This notebook comes with a range of incomplete code-cells.  
Whenever there is something to do for you, you will see this marker:

## :book: EXCERCISE

This means the next - **and only the next** - code-cell needs your expertise.  ____

To make it even easier, every cell that needs to be changed can be found in navigation on the right.  
They start with **___ EXCERCISE ___** followed by a number.

# Imports

In [ ]:
# Snowpark Imports
from snowflake.snowpark.context import get_active_session
import snowflake.snowpark.functions as F
from snowflake.snowpark.functions import col, lit, when

# Snowpark ML
from snowflake.ml.modeling.impute import SimpleImputer
from snowflake.ml.modeling.preprocessing import OrdinalEncoder, OneHotEncoder, Normalizer
from snowflake.ml.modeling.pipeline import Pipeline
from snowflake.ml.modeling.xgboost import XGBClassifier
from snowflake.ml.modeling.model_selection import GridSearchCV
from snowflake.ml.registry import Registry
from snowflake.ml.feature_store import FeatureStore, FeatureView, CreationMode
from snowflake.cortex import Complete

# Other Imports
import matplotlib.pyplot as plt
import json
import streamlit as st
import plotly.express as px
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

# 2 - Set Up Environment

In [ ]:
# Retrieve the Session
session = get_active_session()

# Set context
session.use_schema('KAGGLE_TITANIC_CHALLENGE.DEVELOPMENT')

# Create reference to Feature Store
fs = FeatureStore(
    session=session, 
    database="KAGGLE_TITANIC_CHALLENGE", 
    name="DEVELOPMENT", 
    default_warehouse="COMPUTE_WH",
    creation_mode=CreationMode.CREATE_IF_NOT_EXIST
)

# Create reference to Model Registry
model_registry = Registry(
    session=session, 
    database_name=session.get_current_database(), 
    schema_name=session.get_current_schema()
)

# 3 - Data Exploration

In [ ]:
entity = fs.get_entity(name="PASSENGER")
kaggle_fv = fs.get_feature_view('PASSENGER_KAGGLE_FEATURES','V1')
custom_fv = fs.get_feature_view('PASSENGER_CUSTOM_FEATURES','V1')

# Retrieve existing features for your data
titanic_df = fs.retrieve_feature_values(session.table('PASSENGER'), [kaggle_fv,custom_fv])
titanic_df.show()

## :book: EXCERCISE  
**Feature Name:**  
AGE

**Description:**  
Visualize the AGE Variable in relation to SURVIVED in a meaningful way.

**Tips:**
* *A [Distribution Plot](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.hist.html) could be useful.*
* *For your convenience I already provided code to generate the required datasets*
    * *passenger_age_survived*
    * *passenger_age_died*

In [ ]:
# Retrieve age values for passengers who survived or died
passenger_age_survived = titanic_df.filter(col('SURVIVED')==1).dropna().select('AGE').to_pandas()['AGE'].tolist()
passenger_age_died = titanic_df.filter(col('SURVIVED')==0).dropna().select('AGE').to_pandas()['AGE'].tolist()


# Plotting the histogram
plt.figure(figsize=(6, 2))
plt.hist([passenger_age_survived, passenger_age_died], 
         bins=10, stacked=True, color=['g', 'r'], label=['SURVIVED', 'DEAD'])
plt.title('Age Histogram by Survival')
plt.xlabel('Age (Years)')
plt.ylabel('# of Passengers')
plt.legend()
plt.show()

# 4 - Feature Engineering

## :book: EXCERCISE 
**Feature Name:**  
IS_ALONE

**Description:**  
Add a new binary feature IS_ALONE that is either 1 or 0.

**Tips:**
* *The feature FAM_SIZE tells you the number of people in the family (1 means the person is alone).*
* *You can use the [iff-function](https://docs.snowflake.com/en/developer-guide/snowpark/reference/python/latest/snowpark/api/snowflake.snowpark.functions.iff) for this.*

In [ ]:
titanic_df = titanic_df.with_column('IS_ALONE', F.iff(col('FAM_SIZE')==1,1,0))

## :book: EXCERCISE 
**Feature Name:**  
AGE_GROUP

**Description:**  
Add a new variable to assign passengers to different age groups.  
If the AGE of a passenger is missing, fill it with the most frequent category.

**Tips:**
* *Use the insight from the average AGE to identify which category to use when the AGE value is missing*
* *You can use the [when-function](https://docs.snowflake.com/en/developer-guide/snowpark/reference/python/latest/snowpark/api/snowflake.snowpark.functions.when) for this.*
* *For your convenience I already provide the conditions that you can utilize*
* *use **.otherwise()** to fill values when AGE is missing* 

In [ ]:
print('Average Age of a passenger is:',titanic_df.select(F.round(F.avg('AGE'),1).as_('AGE')).collect()[0]['AGE'])

condition_child    = (col('AGE') >  0) & (col('AGE') <= 12)
condition_teenager = (col('AGE') > 12) & (col('AGE') <= 18)
condition_adult    = (col('AGE') > 18) & (col('AGE') <= 50)
condition_senior   = (col('AGE') > 50)
condition_missing  = (col('AGE').is_null())

titanic_df = titanic_df.with_column(
    'AGE_GROUP', 
    when(condition_child,"CHILD")
    .when(condition_teenager,"TEENAGER")
    .when(condition_adult,"ADULT")
    .when(condition_senior,"SENIOR")
    .when(condition_missing, 'ADULT'))

## :book: EXCERCISE 
**Description:**  
We are lazy again and don't want to write our own feature descriptions.  
Adjust the prompt to generate a feature description for the columns **IS_ALONE** and **AGE_GROUP** that you created earlier.

**Tips:**
* *Keep it simple! The prompt in general works if the columns are in the dataframe.*

In [ ]:
llm = 'llama3-70b'

prompt = f"""
You are provided with a SQL Query that derives features from existing columns.
Describe the features IS_ALONE and AGE_GROUP.
The descriptions will be stored in a feature store, so make sure to return a JSON where the feature name is the key and the description is the value.
{titanic_df.queries['queries'][0]}
"""

llm_response = Complete(llm, prompt)
feature_descriptions = json.loads(llm_response.split('```')[1])
for key in feature_descriptions:
    feature_descriptions[key] = feature_descriptions[key].replace("'", '')
feature_descriptions

## :book: EXCERCISE 
**Feature View Name:**  
PASSENGER_CUSTOM_FEATURES_2

**Description:**  
Add a new feature view to the Feature Store called PASSENGER_CUSTOM_FEATURES_2

**Tips:**
* *You already have access to the **entity** variable.*
* *Make sure to only add columns to this feature view that **you** created.*
* *Make sure to include the **PASSENGER_ID** variable*

In [ ]:
# Create Feature View with Custom Features
participant_fv = FeatureView(
    name="PASSENGER_CUSTOM_FEATURES_PARTICIPANT", 
    entities=[entity],
    feature_df=titanic_df['PASSENGER_ID','AGE_GROUP','IS_ALONE'],
    refresh_freq="1 minute",
    desc="My awesome new features")

# Add descriptions for some features
participant_fv = participant_fv.attach_feature_desc(feature_descriptions)

participant_fv = fs.register_feature_view(
    feature_view=participant_fv, 
    version="V1", 
    block=True)

In [ ]:
spine_df = session.table('PASSENGER')

spine_df_train = spine_df.filter(col('SURVIVED').is_not_null())
print(f'Train dataset has {spine_df_train.count()} passengers.')
spine_df_train.show(3)

spine_df_test = spine_df.filter(col('SURVIVED').is_null())
print(f'Test dataset has {spine_df_test.count()} passengers.')
spine_df_test.show(3)

## :book: EXCERCISE 
**Description:**  
Generate your Training Dataset.  
Don't forget to utilize all feature views:
* kaggle_fv, custom_fv and participant_fv

**Tips:**
* *You already have access to the **entity** variable.*
* *Make sure to only add columns to this feature view that **you** created.*
* *Make sure to include the **PASSENGER_ID** variable*

In [ ]:
# Generate the training dataset by retrieving the features
training_dataset = fs.generate_dataset(
    name="TITANIC_TRAINING_DATASET",
    spine_df=spine_df_train,
    features=[kaggle_fv,custom_fv,participant_fv],
    spine_label_cols=["SURVIVED"],
    desc="Training Data to train model to predict whether a passenger survived."
)

# Retrieve a Snowpark DataFrame from the registered Dataset
training_dataset_df = training_dataset.read.to_snowpark_dataframe().cache_result()
training_dataset_df.show(3)

# 5 - Modelling

In [ ]:
# Switch to a larger warehouse to speed up training
session.use_warehouse('TRAIN_WH')

## :book: EXCERCISE 
**Description:**  
Adjust the preprocessing pipeline to include your newly created columns IS_ALONE and AGE_GROUP.

**Tips:**
* *AGE_GROUP is an ordinal variable. Make sure you configure an [OrdinalEncoder](https://docs.snowflake.com/en/developer-guide/snowpark-ml/reference/latest/api/modeling/snowflake.ml.modeling.preprocessing.OrdinalEncoder).*

In [ ]:
# DROP unused variables
# PASSENGER_ID -> just an artificial ID with no predictive value
# NAME -> people won't survive just because of their name and we extracted the title already
# CABIN -> too many missing values
# TICKET -> doesn't contain valuable information in its current form since it's a unique value per customer
training_dataset_df = training_dataset_df.drop(['PASSENGER_ID','NAME','CABIN','TICKET'])

# Impute Age by mean
si_age =  SimpleImputer(
    input_cols=['AGE','FARE'], 
    output_cols=['AGE_IMP','FARE_IMP'],
    strategy='mean',
    drop_input_cols=True
)

# ***************************************
# ** DEFINE YOUR ORDINAL-ENCODER here  **
# ** vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv  **
# ***************************************
# Define categories for the OrdinalEncoder
categories = {
        "AGE_GROUP": np.array(["CHILD", "TEENAGER", "ADULT", "SENIOR"]),
    }

oe = OrdinalEncoder(
    input_cols=["AGE_GROUP"], 
    output_cols=["AGE_GROUP_OE"], 
    categories=categories,
    drop_input_cols=True
)
# ***************************************
# ** ɅɅɅɅɅɅɅɅɅɅɅɅɅɅɅɅɅɅɅɅɅɅɅɅɅɅɅɅɅɅɅɅɅ  **
# ** DEFINE YOUR ORDINAL-ENCODER here  **
# ***************************************

# Normalize Fare and Age
norm = Normalizer(
    input_cols=['AGE_IMP','FARE_IMP'],
    output_cols=['AGE_IMP_NORM','FARE_IMP_NORM'],
    drop_input_cols=True
)

# One-Hot-Encoding of categorical features
ohe = OneHotEncoder(
    input_cols=['SEX','EMBARKED','TITLE','FAM_SIZE_CATEGORY'], 
    output_cols=['SEX','EMBARKED','TITLE','FAM_SIZE_CATEGORY'],
    drop_input_cols=True
)

# Define the XGBoost model (incl. Hyperparameter Tuning)
label_cols = ['SURVIVED']
output_cols = ['SURVIVED_PREDICTION']

grid_search = GridSearchCV(
    estimator=XGBClassifier(random_state=42),
    param_grid={
        'n_estimators':[10, 50, 100],
        'max_depth': [2,4,8],
        'learning_rate':[.01, .03, .1],
    },
    n_jobs = -1,
    scoring="accuracy",
    label_cols=label_cols,
    output_cols=output_cols
)

# Build the pipeline
model_pipeline = Pipeline(
    steps=[
        ("IMPUTE",si_age),
        ("ORDINAL_ENCODE",oe),
        ("NORMALIZE",norm),
        ("ONE_HOT_ENCODE",ohe),
        ("GRIDSEARCH_XGBOOST",grid_search)
    ]
)

# Fit the pipeline to the training data
fitted_pipeline = model_pipeline.fit(training_dataset_df)

In [ ]:
# Switch back to smaller warehouse to save ressources
session.use_warehouse('COMPUTE_WH')

# 6 - Model Evaluation

In [ ]:
model_object = fitted_pipeline.to_sklearn().named_steps['GRIDSEARCH_XGBOOST']

# Get the Feature Importance
feature_importance = pd.DataFrame(
    zip(model_object.best_estimator_.feature_names_in_, model_object.best_estimator_.feature_importances_),
    columns=['FEATURE_NAME','IMPORTANCE']
).sort_values('IMPORTANCE', ascending=False)

fig = px.bar(feature_importance, x='FEATURE_NAME', y='IMPORTANCE')
st.plotly_chart(fig)

In [ ]:
# Build a dataframe for the gridsearch results
gs_results = model_object.cv_results_
n_estimators_val = []
learning_rate_val = []
max_depth_val = []
for param_dict in gs_results["params"]:
    n_estimators_val.append(param_dict["n_estimators"])
    learning_rate_val.append(param_dict["learning_rate"])
    max_depth_val.append(param_dict["max_depth"])
accuracy_val = gs_results["mean_test_score"]

gs_results_df = pd.DataFrame(data={
    "n_estimators":n_estimators_val,
    "learning_rate":learning_rate_val,
    "max_depth":max_depth_val,
    "accuracy":accuracy_val})

print(f'Number of Models: {len(gs_results_df)}')
print('Best Parameter Configuration:')
model_object.best_params_
print(f'Accuracy of best Model: {model_object.best_score_}')

# Create a 3D scatter plot to visualize impact of parameters on accuracy
fig = px.scatter_3d(gs_results_df, x='learning_rate', y='n_estimators', z='max_depth', color='accuracy',
                    labels={'accuracy': 'accuracy'})

# Update the layout to increase the size of the chart
fig.update_layout(
    width=1000,  # Set the desired width
    height=800   # Set the desired height
)

# Display the chart
st.plotly_chart(fig)

# 7 - Register Model

In [ ]:
# Register new model version
registered_model = model_registry.log_model(
    fitted_pipeline,
    model_name="TITANIC_SURVIVAL_MODEL",
    comment="Model trained using GridsearchCV in Snowpark to predict survival of Titanic passengers.",
    metrics={"accuracy": model_object.best_score_},
    conda_dependencies=['xgboost'],
    version_name='PARTICIPANT_VERSION'
)

# View available models
model_registry.show_models()

# View available versions
model_registry.get_model('TITANIC_SURVIVAL_MODEL').show_versions()

# 8 - Scoring Passengers

## :book: EXCERCISE 
**Description:**  
Generate your Test Dataset.  
Don't forget to utilize all feature views:
* kaggle_fv, custom_fv and participant_fv

**Tips:**
* *You already have access to the **entity** variable.*

In [ ]:
# Retrieving features for the test data
test_dataset_df = fs.retrieve_feature_values(
    spine_df=spine_df_test, 
    features=[kaggle_fv,custom_fv,participant_fv], 
    exclude_columns=['SURVIVED']
)
test_dataset_df.show(3)

In [ ]:
# Create and persist predictions from registered model given the retrieved features
predictions = registered_model.run(test_dataset_df, function_name='predict')
predictions.write.save_as_table('TITANIC_TEST_PREDICTIONS', mode='overwrite')

# 
predictions = session.table('TITANIC_TEST_PREDICTIONS')
predictions.show()

# 9 - Score your results!
Kaggle expects a CSV file with two columns:  
* PassengerId
* Survived

After offloading the data as CSV-file, you can download it via Snowflake's UI.

If you don't have a Kaggle Account, you can also evaluate your performance with the following call:  
**session.call('calculate_challenge_score', <path-to-your-csv-file>)**


In [ ]:
# Transform data into format expected by Kaggle
kaggle_submission = predictions.select('PASSENGER_ID','SURVIVED_PREDICTION')
kaggle_submission = kaggle_submission.with_column_renamed(col('PASSENGER_ID'),'"PassengerId"')
kaggle_submission = kaggle_submission.with_column_renamed(col('SURVIVED_PREDICTION'),'"Survived"')
kaggle_submission.show()

# Export Predictions to submission.csv
kaggle_submission.write.csv(
    '@KAGGLE_SUBMISSION/submission_participant.csv', 
    header=True, 
    single=True, 
    format_type_options={"COMPRESSION": "NONE"},
    overwrite=True
)

In [ ]:
session.call('calculate_challenge_score', '@KAGGLE_SUBMISSION/submission_participant.csv')

## :book: EXCERCISE 
**Description:**  
Finally compare your model with the model from the demo.  
Were you able to improve the model accuracy?

**Tips:**
* *You can easily retrieve saved metrics from your models by using **get_metric('accuracy')**.*

In [ ]:
demo_version_accuracy = model_registry.get_model('TITANIC_SURVIVAL_MODEL').version('DEMO_VERSION').get_metric('accuracy')
participant_version_accuracy = model_registry.get_model('TITANIC_SURVIVAL_MODEL').version('PARTICIPANT_VERSION').get_metric('accuracy')

print(f'Demo Version Accuracy: {demo_version_accuracy}')
print(f'Participant Version Accuracy: {participant_version_accuracy}')